In [ ]:
import numpy as np

# 1. Инкапсуляция
Под инкапсуляцией в объектно-ориентированном программировании понимается упаковка данных и методов для их обработки вместе, т. е. в классе. В Python инкапсуляция реализуется как на уровне классов, так и объектов. В ряде других языков, например в Java, под инкапсуляцией также понимают сокрытие свойств и методов, в результате чего они становятся приватными. Это значит, что доступ к ним ограничен либо пределами класса, либо модуля.

В `Python` подобной инкапсуляции нет, хотя существует способ ее имитировать. Перед тем как выяснять, как это делается, надо понять, зачем вообще что-то скрывать.

Дело в том, что классы бывают большими и сложными. В них может быть множество вспомогательных полей и методов, которые не должны использоваться за его пределами. Они просто для этого не предназначены. Они своего рода внутренние шестеренки, обеспечивающие нормальную работу класса.

Кроме того, в других языках программирования хорошей практикой считается сокрытие всех полей объектов, чтобы уберечь их от прямого присвоения значений из основной ветки программы. Их значения можно изменять и получать только через вызовы методов, специально определенных для этих целей.

Например, если надо проверять присваиваемое полю значение на корректность, то делать это каждый раз в основном коде программы будет неправильным. Проверочный код должен быть помещен в метод, который получает данные, для присвоения полю. А само поле должно быть закрыто для доступа из вне класса. В этом случае ему невозможно будет присвоить недопустимое значение.



Например, если класс имеет счетчик своих объектов, то необходимо исключить возможность его случайного изменения извне. Рассмотрим пример:

In [ ]:
class B:
    count = 0
    def __init__(self):
        B.count += 1
    def __del__(self):
        B.count -= 1

In [ ]:
a = B()
b = B()
с = B()
print(b.count)
print(B.count) 
del a
print(B.count) 

In [ ]:
b.count

In [ ]:
B.count -= 1
print(B.count)

Для *имитации* сокрытия атрибутов в Python используется соглашение (соглашение – это не синтаксическое правило языка, при желании его можно нарушить), согласно которому, если поле или метод имеют два знака подчеркивания впереди имени, но не сзади, то этот атрибут предусмотрен исключительно для внутреннего пользования:

In [ ]:
class B:
    __count = 0
 
    def __init__(self):
        B.__count += 1
 
    def __del__(self):
        B.__count -= 1
 
 
a = B()
print(B.__count)

In [ ]:
B.__count = 3
B.__count

На самом деле сокрытие в `Python` не настоящее и доступ к счетчику мы получить все же можем. 

In [ ]:
print(B._B__count)

А как теперь вернуть значение?

In [ ]:
class B:
    __count = 0
 
    def __init__(self):
        B.__count += 1
 
    def __del__(self):
        B.__count -= 1
    
    def qty_objects():# создали метод для этой цели
        return B.__count
            
a = B()
b = B()
print(B.qty_objects())

#$B.qty_objects()
#a.__count 

В `Python` атрибуты объекту можно назначать за пределами класса:

In [ ]:
class A:
     def __init__(self, v):
        self.field1 = v
a = A(10)
a.field2 = 20
a.field1,a.field2


Если это поведение нежелательно, то можем использовать метод `__setattr__`

In [ ]:
class A:
    def __init__(self, v):
        self.field1 = v
    def __setattr__(self, attr, value):
        if attr == 'field1':
            self.__dict__[attr] = value
        else:
            raise AttributeError
a = A(15)
a.field1       

In [ ]:
a.__dict__

In [ ]:
 a.field2 = 30

Метод `__setattr__()`, если он присутствует в классе, вызывается всегда, когда какому-либо атрибуту выполняется присваивание. !!!!Обратите внимание, что присвоение несуществующему атрибуту также обозначает его добавление к объекту.

Когда создается объект `a1`, в конструктор передается число `15`. Здесь для объекта заводится атрибут `field1`. Факт попытки присвоения ему значения тут же отправляет интерпретатор в метод `__setattr__()`, где проверяется соответствует ли имя атрибута строке `field1`. Если так, то атрибут и соответствующее ему значение добавляется в словарь атрибутов объекта.

Нельзя в `__setattr__()` написать просто `self.field1 = value`, так как это приведет к новому рекурсивному вызову метода `__setattr__()`. Поэтому поле назначается через словарь `__dict__`, который есть у всех объектов, и в котором хранятся их атрибуты со значениями.

Если параметр `attr` не соответствует допустимым полям, то искусственно возбуждается исключение `AttributeError`. Мы это видим, когда в основной ветке пытаемся обзавестись полем `field2`.

Если объект содержит скрытые поля и к ним происходит обращение из `__setattr__()`, то делать это надо так, как будто обращение происходит не из класса. Следующий код приведет к генерации исключения:

In [ ]:
class A:
    def __init__(self, x):
        self.__x = x
 
    def __setattr__(self, attr, value):
        if attr == "__x":
            self.__dict__[attr] = value
        else:
            raise AttributeError

a = A(5)

Более подробно про `setattr`   [тут](https://datagy.io/python-setattr/)

# 2. Полиморфизм
Полиморфизм в объектно-ориентированном программировании – это возможность обработки разных типов данных, т. е. принадлежащих к разным классам, с помощью "одной и той же" функции, или метода. На самом деле одинаковым является только имя метода, его исходный код зависит от класса. Кроме того, результаты работы одноименных методов могут существенно различаться. Поэтому в данном контексте под полиморфизмом понимается множество форм одного и того же слова – имени метода.


Еще раз всмомним пример из прошлого семинара


Например, два разных класса содержат метод `total`, однако инструкции каждого предусматривают совершенно разные операции. Так в классе `T1` – это прибавление `10` к аргументу, в `T2` – подсчет длины строки символов. В зависимости от того, к объекту какого класса применяется метод `total`, выполняются те или иные инструкции.

In [ ]:
class T1:
    n = 10
    def total(self, N):
        self.total = int(self.n) + int(N)

class T2:
    def total(self,s):
        self.total = len(str(s))

In [ ]:
t1 = T1()
t2 = T2()
t1.total(45)
t2.total(45)
print(t1.total) 
print(t2.total) 

Как видно, полиморфизмом обладают классы связанные наследованием. У каждого может быть свой метод `__init__()`, или `square()`, или какой-нибудь другой. Какой именно из методов `square()` вызывается и что он делает, зависит от принадлежности объекта к тому или иному классу.

Однако классы не обязательно должны быть связаны наследованием. Полиморфизм как один из ключевых элементов ООП существует независимо от наследования. Классы могут быть не родственными, но иметь одинаковые методы.

Полиморфизм дает возможность реализовывать так называемые единые интерфейсы для объектов различных классов. Например, разные классы могут предусматривать различный способ вывода той или иной информации объектов. Однако одинаковое название метода вывода позволит не запутать программу, сделать код более ясным.



Рассмотрим пример полиморфизма на еще одном методе, который перегружает функцию `print()`.

Если вы создадите объект собственного класса, а потом попробуете вывести его на экран, то получите информацию о классе объекта и его адрес в памяти. Такое поведение функции `print()` по умолчанию по отношению к пользовательским классам запрограммировано на самом верхнем уровне иерархии, где-то в суперклассе, от которого неявно наследуются все остальные.

In [ ]:
class A:
    def __init__(self, v1, v2):
        self.field1 = v1
        self.field2 = v2

In [ ]:
a = A(3, 4)
print(a)

Если же мы хотим, чтобы, когда объект передается функции `print()`, выводилась какая-нибудь другая более полезная информация, то в класс надо добавить специальный метод `__str__()`. Этот метод должен обязательно возвращать строку, которую будет выводить функция `print()`:

In [ ]:
class A:
    def __init__(self, v1, v2):
        self.field1 = v1
        self.field2 = v2
    def __str__(self):
        return str(self.field1) + " " + str(self.field2)

In [ ]:
a = A(3, 4)
print(a)

# 3. Наследование
Наследование – важная составляющая объектно-ориентированного программирования. Так или иначе мы уже сталкивались с ним, ведь объекты наследуют атрибуты своих классов. Однако обычно под наследованием в ООП понимается наличие классов и подклассов. Также их называют супер- или надклассами и классами, а также родительскими и дочерними классами.

Суть наследования здесь схожа с наследованием объектами от классов. Дочерние классы наследуют атрибуты родительских, а также могут переопредять атрибуты и добавлять свои.

## 3.1. Простое наследование

In [ ]:
class Table:
    def __init__(self, l, w, h):
        self.lenght = l
        self.width = w
        self.height = h

In [ ]:
class KitchenTable(Table):
    def setPlaces(self, p):
        self.places = p

In [ ]:
 class DeskTable(Table):
    def square(self):
        return self.width * self.lenght

В данном случае классы `KitchenTable` и `DeskTable` не имеют своих собственных конструкторов, поэтому наследуют его от родительского класса. При создании экземпляров этих столов, передавать аргументы для `__init__()` обязательно, иначе возникнет ошибка:

In [ ]:
t1 = KitchenTable()

In [ ]:
t1 = KitchenTable(2, 2, 0.7)
t2 = DeskTable(1.5, 0.8, 0.75)
t3 = KitchenTable(1, 1.2, 0.8)

In [ ]:
t3.lenght

Можно создавать столы и от родительского класса `Table`. Однако он не будет, согласно неким родственным связям, иметь доступ к методам `setPlaces()` и `square()`. Точно так же объект класса `KitchenTable` не имеет доступа к единоличным атрибутам сестринского класса `DeskTable`:

In [ ]:
t4 = Table(1, 1, 0.5)

In [ ]:
t2.width * t2.lenght

In [ ]:
t2.square()

In [ ]:
t4.square()

In [ ]:
t3.square()

## 3.2. Полное переопределение метода надкласса
Что, если в подклассе нам не подходит код метода его надкласса? Допустим, мы вводим еще один класс столов, который является дочерним по отношению к `DeskTable`. Пусть это будут компьютерные столы, при вычислении рабочей поверхности которых надо отнимать заданную величину. Имеет смысл внести в этот новый подкласс его собственный метод `square()`:

In [ ]:
class ComputerTable(DeskTable):
    def square(self, e):
        return self.width * self.lenght - e

При создании объекта типа `ComputerTable` по-прежнему требуется указывать параметры, так как интерпретатор в поисках конструктора пойдет по дереву наследования сначала в родителя, а потом в прародителя и найдет там метод `__init__()`.

Однако когда будет вызываться метод `square()`, то поскольку он будет обнаружен в самом `ComputerTable`, то метод `square()` из `DeskTable` останется невидимым, т. е. для объектов класса `ComputerTable` он окажется переопределенным.

In [ ]:
ct = ComputerTable(2, 1, 1)
ct.square(0.3)

Или:

In [ ]:
class ComputerTable(DeskTable):
    def square(self, e):
        return DeskTable.square(self) - e 

In [ ]:
ct = ComputerTable(2, 1, 1)
ct.square(0.3)

Допустим, в классе `KitchenTable` нам не нужен метод, поле `places` должно устанавливаться при создании объекта в конструкторе. В классе можно создать собственный конструктор с чистого листа, чем переопределить родительский:

In [ ]:
class KitchenTable(Table):
    def __init__(self, l, w, h, p):
        self.length = l
        self.width = w
        self.height = h
        self.places = p

Однако если дублируется почти весь конструктор надкласса, проще вызвать родительский конструктор, после чего дополнить своим кодом:

In [ ]:
class KitchenTable(Table):
    def __init__(self, l, w, h, p):
        Table.__init__(self, l, w, h)
        self.places = p

In [ ]:
tk = KitchenTable(2, 1.5, 0.7, 10)

In [ ]:
tk.places

In [ ]:
tk.width 

## 3.3. Множественное наследование

Для наследования от нескольких классов в определении класса-наследника достаточно указать в круглых скобках сразу несколько базовых классов через запятую.

# Пример.

Рассмотрим три класса:

In [ ]:
class Class1: 
    def funс1(self):
        print("Метод funс1() класса Classl")

class Class2(Class1): # Простое наследование
    def func2(self):
        print("Метод func2() класса Class2")

class Class3(Class1): # Простое наследование
    def funс1(self):
        print("Метод funс1() класса Class3")
    def func2(self):
        print("Метод func2() класса Class3")
    def func3(self):
        print("Метод func3() класса Class3")
    def func4(self):
        print("Метод func4() класса Class3")

class Class4(Class2, Class3): # Множественное наследование
    def func4(self):
        print("Метод func4() класса Class4")

In [ ]:
c = Class4()
c.funс1() 
c.func2() 
c.func3() 
c.func4() 

Метод `func1()` определен в двух классах: `class1` и `class3`. Так как вначале просматриваются все базовые классы, непосредственно указанные в определении текущего класса, метод `func1()` будет найден в классе `class3` (поскольку он указан в числе базовых классов в определении `Class4`), а не в классе `Class1`.

Метод `func2()` также определен в двух классах: `Class2` и `Class3`. Так как класс `Class2` стоит первым в списке базовых классов, то метод будет найден именно в нем. 

Чтобы наследовать метод из класса `Class3`, следует указать это явным образом:

In [ ]:
class Class4(Class2, Class3): # Множественное наследование
# Наследуем func2() из класса Class3, а не из класса Class2
    func2 = Class3.func2
    def func4(self):
        print("Метод func4() класса Class4")

In [ ]:
c = Class4()
c.funс1() 
c.func2() 
c.func3() 
c.func4() 

# 4. Композиция

Еще одной особенностью объектно-ориентированного программирования является возможность реализовывать так называемый композиционный подход. Заключается он в том, что есть класс-контейнер, он же агрегатор, который включает в себя вызовы других классов. В результате чего, что при создании объекта класса-контейнера, также создаются объекты включенных в него классов.

Не следует путать композицию с наследованием, в том числе множественным. Наследование предполагает принадлежность к какой-то общности (похожесть), а композиция — формирование целого из частей. Наследуются атрибуты, т.е. возможности, другого класса, при этом объектов непосредственно родительского класса не создается. При композиции же класс-агрегатор создает объекты других классов.

Рассмотрим на примере реализацию композиции в Python. Пусть, требуется написать программу, которая вычисляет площадь стен. При этом окна, двери, пол и потолок должны быть исключены.

Итак, комната – это прямоугольный параллелепипед, состоящий из шести прямоугольников. Его площадь представляет собой сумму площадей составляющих его прямоугольников. Площадь прямоугольника равна произведению его длины на ширину.

In [ ]:
class WinDoor:
     def __init__(self, x, y):
        self.square = x * y 

In [ ]:
class Room:
    def __init__(self, x, y, z):
        self.square = 2 * z * (x + y)
        self.wd = []
    def addWD(self, w, h):
        self.wd.append(WinDoor(w, h))
    def workSurface(self):
        new_square = self.square
        for i in self.wd:
            new_square -= i.square
        return new_square

In [ ]:
r1 = Room(6, 3, 2.7) 
print(r1.square)
r1.addWD(1, 1) 
r1.addWD(1, 1)
r1.addWD(1, 2)
print(r1.workSurface())

# Задачи

## Упражнение 1

Измените класс `A` таким образом, чтобы объект этого класса мог содержать произвольное число параметров. Перегрузите для него функции `__bool__()` и `__len__()`.

In [ ]:
class A:
    def __init__(self, *args):
        self.args = list(args)
    def __str__(self):
        return " ".join(str(arg) for arg in self.args)
    # Не понял какое поведение ожидается от bool    
    def __bool__(self):
        return bool(self.args)
        
    def __len__(self):
        return len(self.args)

In [ ]:
test = A("hello", "world")
print(test)
print(bool(test))
print(len(test))

## Упражнение 2

Напишите класс `Numbers`, содержащий два атрибута `x` и `y`, функцию инициализации и функции `Operation`(пусть будет умножение 2ух чисел) и `Run`(калькуляция этой функций):

In [ ]:
class Numbers:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.result  = lambda result:f"{self.x} * {self.y} = {result}"
    
    def Operation(self):
        return self.x * self.y
    
    def Run(self):
        result = self.Operation()
        print(self.result(result))
        return result

Создайте класс-наследник и с помощью полиморфизма замените функцию `Operation` на любую другую в наследном классе.

In [ ]:
class MegaNumbers(Numbers):
    def __init__(self, x, y):
        super().__init__(x, y)
        self.result = lambda result:f"{self.x} ** {self.y} = {result}"
    def Operation(self):
        return self.x ** self.y

In [ ]:
test = Numbers(2,3)
test.Run()
test = MegaNumbers(2,3)
test.Run()

## Упражнение 3

Дополните приведённый пример возможностью вычисления площади пола и потолка, при этом в полу может быть квадратный проём с лестницей, ведущей вниз, а на потолке – круглые окна.

In [ ]:
class WinDoor:
     def __init__(self, x, y):
        self.square = x * y 
class Room:
    def __init__(self, x, y, z):
        self.square = 2 * z * (x + y)
        self.wd = []
    def addWD(self, w, h):
        self.wd.append(WinDoor(w, h))
    def workSurface(self):
        new_square = self.square
        for i in self.wd:
            new_square -= i.square
        return new_square

In [ ]:
r1 = Room(6, 3, 2.7) 
print(r1.square)
r1.addWD(1, 1) 
r1.addWD(1, 1)
r1.addWD(1, 2)
print(r1.workSurface())

Решение

In [ ]:
class WinDoor:
    def __init__(self, x, y):
        self.square = x * y
        
class Room:
    def __init__(self, x, y, z):
        self.width = x
        self.length = y
        self.height = z
        self.wd = []
        
    def addWD(self, w, h):
        self.wd.append(WinDoor(w, h))
        
    def workSurface(self):
        floor_square = self.width * self.length
        ceiling_square = floor_square
        
        for i in self.wd:
            # Если проём находится в полу, вычитаем его площадь из площади пола
            if i.square <= floor_square:
                floor_square -= i.square
            # Если проём находится в потолке, вычитаем его площадь из площади потолка
            elif i.square <= ceiling_square:
                ceiling_square -= i.square
                
        return (floor_square, ceiling_square)

In [ ]:
r1 = Room(6, 3, 2.7) 
r1.addWD(1, 1) 
r1.addWD(1, 1)
r1.addWD(1, 2)
floor_square, ceiling_square = r1.workSurface()
print(f"Площадь пола: {floor_square}")
print(f"Площадь потолка: {ceiling_square}")

## Задача 1
Класс Mentor должен стать родительским классом, а от него нужно реализовать наследование классов Lecturer (лекторы) и Reviewer (эксперты, проверяющие домашние задания)

In [ ]:
class Student:
    def __init__(self, name, surname, gender):
        self.name = name
        self.surname = surname
        self.gender = gender
        self.finished_courses = []
        self.courses_in_progress = []
        self.grades = {}
        
class Mentor:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        self.courses_attached = []
        
    def rate_hw(self, student, course, grade):
        if isinstance(student, Student) and course in self.courses_attached and course in student.courses_in_progress:
            if course in student.grades:
                student.grades[course] += [grade]
            else:
                student.grades[course] = [grade]
        else:
            return 'Ошибка'
 
best_student = Student('Ruoy', 'Eman', 'your_gender')
best_student.courses_in_progress += ['Python']
 
cool_mentor = Mentor('Some', 'Buddy')
cool_mentor.courses_attached += ['Python']
 
cool_mentor.rate_hw(best_student, 'Python', 10)
cool_mentor.rate_hw(best_student, 'Python', 10)
cool_mentor.rate_hw(best_student, 'Python', 10)
 
print(best_student.grades)

Решение

In [ ]:
class Student:
    def __init__(self, name, surname, gender):
        self.name = name
        self.surname = surname
        self.gender = gender
        self.finished_courses = []
        self.courses_in_progress = []
        self.grades = {}
        
class Mentor:
    def __init__(self, name, surname):
        self.name = name
        self.surname = surname
        self.courses_attached = []
        
    def rate_hw(self, student, course, grade):
        if isinstance(student, Student) and course in self.courses_attached and course in student.courses_in_progress:
            if course in student.grades:
                student.grades[course] += [grade]
            else:
                student.grades[course] = [grade]
        else:
            return 'Ошибка' # будем хорошие и не будем кидать exception

class Lecturer(Mentor):
    def __init__(self, name, surname):
        super().__init__(name, surname)
        self.courses_attached = []
        self.grades = {}

class Reviewer(Mentor):
    def __init__(self, name, surname):
        super().__init__(name, surname)
        self.courses_attached = []

In [ ]:
best_student = Student('Ruoy', 'Eman', 'your_gender')
best_student.courses_in_progress += ['Python']
 
cool_mentor = Mentor('Some', 'Buddy')
cool_mentor.courses_attached += ['Python']
 
cool_mentor.rate_hw(best_student, 'Python', 10)
cool_mentor.rate_hw(best_student, 'Python', 10)
cool_mentor.rate_hw(best_student, 'Python', 10)
 
print(best_student.grades)

best_lecturer = Lecturer('John', 'Doe')
best_lecturer.courses_attached += ['Python']
 
best_lecturer.rate_hw(best_student, 'Python', 10)
best_lecturer.rate_hw(best_student, 'Python', 9)
 
print(best_student.grades)

best_reviewer = Reviewer('Jane', 'Doe')
best_reviewer.courses_attached += ['Python']
 
best_reviewer.rate_hw(best_student, 'Python', 10)
best_reviewer.rate_hw(best_student, 'Python', 8)
 
print(best_student.grades)

## Задача 2
Реализуйте метод выставления оценок лекторам у класса Student (оценки по 10-балльной шкале, хранятся в атрибуте-словаре у Lecturer, в котором ключи – названия курсов, а значения – списки оценок). Лектор при этом должен быть закреплен за тем курсом, на который записан студент.

In [ ]:
class Student2(Student):
    def rate_lecturer(self, lecturer, course, grade):
        if course in self.courses_in_progress and course in lecturer.courses_attached:
            if course in lecturer.grades:
                lecturer.grades[course] += [grade]
            else:
                lecturer.grades[course] = [grade]
        else:
            return 'Ошибка'

In [ ]:

best_student = Student2('Ruoy', 'Doy', 'мускулинный бисексуал гендер-флюид') # Американцы, что с них взять
best_student.courses_in_progress += ['Python']

cool_lecturer = Lecturer('Some', 'Buddy')
cool_lecturer.courses_attached += ['Python']

best_student.rate_lecturer(cool_lecturer, 'Python', 10)
best_student.rate_lecturer(cool_lecturer, 'Python', 9)
best_student.rate_lecturer(cool_lecturer, 'Python', 8)

print(cool_lecturer.grades)

## Задача 3

Реализуйте возможность сравнивать (через операторы сравнения) между собой лекторов по средней оценке за лекции и студентов по средней оценке за домашние задания.

In [ ]:
class Lecturer2(Lecturer):
    def __init__(self, name, surname):
        super().__init__(name, surname)

    def __str__(self):
        return f"{self.name} {self.surname}"

    def __lt__(self, other):
        return self.get_avg_lecture_grade() < other.get_avg_lecture_grade()

    def __eq__(self, other):
        return self.get_avg_lecture_grade() == other.get_avg_lecture_grade()

    def get_avg_lecture_grade(self):
        total_grade = 0
        count = 0
        for grades in self.grades.values():
            total_grade += sum(grades)
            count += len(grades)
        return total_grade / count if count != 0 else 0


class Student3(Student2):

    def __str__(self):
        return f"{self.name} {self.surname}"

    def __lt__(self, other):
        return self.get_avg_hw_grade() < other.get_avg_hw_grade()

    def __eq__(self, other):
        return self.get_avg_hw_grade() == other.get_avg_hw_grade()

    def get_avg_hw_grade(self):
        total_grade = 0
        count = 0
        for grades in self.grades.values():
            total_grade += sum(grades)
            count += len(grades)
        return total_grade / count if count != 0 else 0

In [ ]:
# Создаем несколько объектов класса Lecturer
lecturer1 = Lecturer2("Иван", "Петров")
lecturer1.courses_attached += ['Python']
lecturer2 = Lecturer2("Петр", "Иванов")
lecturer2.courses_attached += ['Python']
lecturer3 = Lecturer2("Сидор", "Сидоров")
lecturer3.courses_attached += ['Python']

# Добавляем студента и выставляем оценки за лекции
best_student = Student3('Ruoy', 'Doy', 'мускулинный бисексуал гендер-флюид')
best_student.courses_in_progress += ['Python']
best_student.rate_lecturer(lecturer1, 'Python', 6)
best_student.rate_lecturer(lecturer2, 'Python', 7)
best_student.rate_lecturer(lecturer3, 'Python', 8)

best_student2 = Student3('Richard', 'Doy', 'Helicopter Apache')
best_student2.courses_in_progress += ['Python']
best_student2.rate_lecturer(lecturer1, 'Python', 9)
best_student2.rate_lecturer(lecturer2, 'Python', 6)
best_student2.rate_lecturer(lecturer3, 'Python', 10)

print(lecturer1.get_avg_lecture_grade())
print(lecturer2.get_avg_lecture_grade())
print(lecturer3.get_avg_lecture_grade())

# Выставляем оценки студентам

lecturer1.rate_hw(best_student, 'Python', 8)
lecturer2.rate_hw(best_student, 'Python', 5)
lecturer3.rate_hw(best_student, 'Python', 10)

lecturer1.rate_hw(best_student2, 'Python', 8)
lecturer2.rate_hw(best_student2, 'Python', 4)
lecturer3.rate_hw(best_student2, 'Python', 10)

print(best_student.get_avg_hw_grade())
print(best_student2.get_avg_hw_grade())

## Задача 4

Напишите класс "Формула" для создания символьной записи формулы и её вычисления. Класс должен содержать атрибуты:

- список операций производимых над переменной "x",
- метод для вывода всех операторов и вывода определённого оператора по его номеру,
- метод для составления формулы как строки скомпанованной из заданных операторов,
- метод для изменения оператора,
- метод для подстановки значения x и вычисления значения формулы (например, с помощью функции `eval()`).

In [ ]:
class Formula:
    def __init__(self):
        self.operations = []
        
    def add_operation(self, operation):
        self.operations.append(operation)
        
    def get_operations(self):
        return self.operations
        
    def get_operation(self, i):
        if i >= len(self.operations) or i < 0:
            raise IndexError("Index out of range")
        return self.operations[i]
        
    def compose_formula(self):
        return ''.join(self.operations)
        
    def set_operation(self, i, operation):
        if i >= len(self.operations) or i < 0:
            raise IndexError("Index out of range")
        self.operations[i] = operation
    
    def clear(self):
        self.operations = []

    def evaluate(self, x):
        formula = self.compose_formula().replace("x", str(x))
        return eval(formula)

In [ ]:
f = Formula()
f.add_operation("x ** 2")
f.add_operation("+")
f.add_operation("10")

print(f.compose_formula()) 

f.set_operation(0, "x * ")
print(f.compose_formula()) 

print(f.evaluate(2)) 

Определите для объектов класса "Формула" алгебраические операции: +, -, *, /

In [ ]:
class Formula2(Formula):
    def __add__(self, other): # Надо было конструктор прикрутить, который бы хавал просто n-ое количество строк и соеденял бы просто, но что-то не сделал
        result = Formula2()
        result.add_operation(self.compose_formula() + " + " + other.compose_formula())
        return result

    def __sub__(self, other):
        result = Formula2()
        result.add_operation(self.compose_formula() + " - " + other.compose_formula())
        return result

    def __mul__(self, other):
        result = Formula2()
        result.add_operation(self.compose_formula() + " * " + other.compose_formula())
        return result

    def __truediv__(self, other):
        result = Formula2()
        result.add_operation(self.compose_formula() + " / " + other.compose_formula())
        return result

In [ ]:
f = Formula2()
f.add_operation("x ** 2")
f2 = Formula2()
f2.add_operation("10")
f3 = f + f2

print(f3.compose_formula()) 

print(f3.evaluate(2)) 

## Задача 5

Реализуйте класс `Matrix`. Он должен содержать:

- Конструктор от списка списков. Гарантируется, что списки состоят из чисел, не пусты и имеют одинаковый размер. Конструктор должен копировать содержимое списка списков, т.е. при изменении списков, от которых была сконструирована матрица, содержимое матрицы изменяться не должно.
- Метод `__str__()`, переводящий матрицу в строку. При этом элементы внутри одной строки должны быть разделены знаками табуляции, а строки — переносами строк. После каждой строки не должно быть символа табуляции и в конце не должно быть переноса строки.
- Метод `size()` без аргументов, возвращающий кортеж вида `(<число строк>, <число столбцов>)`. 
- `__add__()`, принимающий вторую матрицу того же размера и возвращающий сумму матриц. В случае, если две матрицы сложить невозможно, должно выводиться сообщение об ошибке.
- `__mul__()`, таким образом, чтобы матрицы можно было умножать на скаляры и на другие матрицы. В случае, если две матрицы перемножить невозможно, должно выводиться сообщение об ошибке.
- `__rmul__()`, делающий то же самое, что и `__mul__()`. Этот метод будет вызван в том случае, когда аргумент находится справа. Для реализации этого метода в коде класса достаточно написать `__rmul__()` = `__mul__()`.
- Статический метод `transposed`, принимающий `Matrix` и возвращающий транспонированную матрицу.

In [ ]:
class Matrix:
    def __init__(self, lst):
        self.matrix = [row.copy() for row in lst]
        
    def __str__(self):
        return '\n'.join(['\t'.join([str(elem) for elem in row]) for row in self.matrix])
        
    def size(self):
        return (len(self.matrix), len(self.matrix[0]))
        
    def __add__(self, other):
        if self.size() != other.size():
            raise Exception("Matrices must have the same size to be added.")
        result = [[self.matrix[i][j] + other.matrix[i][j] for j in range(len(self.matrix[0]))] for i in range(len(self.matrix))]
        return Matrix(result)
    
    def __mul__(self, other):
        if isinstance(other, int) or isinstance(other, float):
            result = [[self.matrix[i][j] * other for j in range(len(self.matrix[0]))] for i in range(len(self.matrix))]
        elif isinstance(other, Matrix):
            if self.size()[1] != other.size()[0]:
                raise Exception("Number of columns in the first matrix must match the number of rows in the second matrix.")
            transposed_other = Matrix.transposed(other) # чтобы было удобно работать с двумя строками
            result = [[sum([self.matrix[i][k] * transposed_other.matrix[j][k] for k in range(self.size()[1])]) for j in range(other.size()[1])] for i in range(self.size()[0])] # топ-10 причин почему я не люблю питон
        else:
            raise Exception("Multiplication is only defined for scalars and matrices.")
        return Matrix(result)
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    @staticmethod
    def transposed(matrix):
        return Matrix([[matrix.matrix[j][i] for j in range(matrix.size()[0])] for i in range(matrix.size()[1])])

In [ ]:
m1 = Matrix([[1, 2], [3, 4]])
m2 = Matrix([[2, 0], [1, 2]])
print(m1 + m2)

In [ ]:
print(m1 * 2)

In [ ]:
m1 = Matrix([[1, 2, 3], [4, 5, 6]])
m2 = Matrix([[1, 2], [3, 4]])
print(m1 * m2)